# 📘 Piranha Trainingsdaten Generator
Dieses Notebook erzeugt Piranha-kompatible Trainingsdaten (einfach anderes Format als dass die Lücken mit Placeholdern gefüllt sind. Inputs sind das golden_dataset_original und die Excel /data/DAiA Manual Labeling.xlsx, Output ist dann /data/piranha_training_data.json

In [23]:
import os
import re
import json
import pandas as pd
from pathlib import Path
import re

GROUPED_PLACEHOLDERS = {
    "NAME": ["TITEL", "VORNAME", "NACHNAME", "SKYPE"],
    "ADRESSE": ["STRASSE", "HAUSNUMMER", "POSTLEITZAHL", "WOHNORT"],
    "VERTRAG": ["VERTRAGSNUMMER", "KUNDENNUMMER", "ZUORDNUNGSNUMMER"],
    "ZAHLUNG": ["ZAHLUNG", "IBAN", "BIC"],
    "TECHNISCHE_DATEN": ["ZÄHLERSTAND", "ZÄHLERNUMMER", "VERBRAUCH", "WLV"],
    "KONTAKT": ["TELEFONNUMMER", "EMAIL", "MAIL", "LINK", "GESENDET_MIT", "FIRMENDATEN", "FAX"],
    "FIRMA": ["FIRMA"],
    "DATUM": ["DATUM"]
}

def map_column_to_label(col_name):
    upper_col = col_name.upper()
    for label, keywords in GROUPED_PLACEHOLDERS.items():
        for keyword in keywords:
            if keyword in upper_col:
                return label
    return None

def generate_piranha_training_data(df, email_folder_path, output_file_path):
    training_data = []

    for index, row in df.iterrows():
        file_name = row["TextFile"]
        email_path = os.path.join(email_folder_path, file_name)

        try:
            with open(email_path, "r", encoding="utf-8") as file:
                text = file.read()
        except FileNotFoundError:
            print(f"Datei nicht gefunden: {file_name}")
            continue

        labels = []
        for col in row.index:
            if pd.isna(row[col]):
                continue
            label_type = map_column_to_label(col)
            if label_type:

                value = str(row[col]).strip()
                if not value:
                 continue  # ← verhindert leere Labels


                # 🔧 Entferne .0 bei Excel-Zahlen
                if value.endswith(".0"):
                    try:
                        value = str(int(float(value)))
                    except ValueError:
                        pass

                # 🔧 Regex-Suchmuster vorbereiten (escaped, tolerant gegenüber Sonderzeichen)
                pattern = re.escape(value)
                match = re.search(pattern, text, re.IGNORECASE)

                if match:
                    start_idx = match.start()
                    end_idx = match.end()
                    labels.append({"start": start_idx, "end": end_idx, "label": label_type})
                else:
                    print(f"Wert nicht gefunden: '{value}' in Datei {file_name}")

        training_data.append({
            "text": text,
            "labels": labels
        })

    with open(output_file_path, "w", encoding="utf-8") as f:
        json.dump(training_data, f, ensure_ascii=False, indent=2)

    print(f"Piranha-Trainingsdaten gespeichert unter: {output_file_path}")



In [ ]:
# 📥 Excel laden und Funktion ausführen
df = pd.read_excel("../data/DAiA Manual Labeling.xlsx", dtype=str)
generate_piranha_training_data(
    df,
    "../../../old_data/golden_dataset_original",
    "../archive/old_data/piranha_old/piranha_training_data.json"
)